In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [34]:
import torch
import cv2
from pathlib import Path
import os
import ultralytics

model = torch.hub.load('ultralytics/yolov5', 'custom', path = '/content/drive/MyDrive/Final_project/image_detect/image_model/yolov5/team2_yolov5m_100.pt')

#탐지 설정 조정
model.imgsz = 416
model.conf = 0.4
model.iou = 0.5
model.agnostic = False
model.stride = 1
model.max_det = 1000
model.line_thickness = 2

save_dir = Path('/content/drive/MyDrive/Final_project/image_detect/image_model/yolov5/result')
crop_dir = save_dir / 'crop' #잘라낸 이미지 저장할 디렉터리
label_dir = save_dir / 'label' #바운딩박스 좌표값 저장할 디렉터리

#디렉터리 없다면 만들기
save_dir.mkdir(parents=True, exist_ok=True)
crop_dir.mkdir(parents=True, exist_ok=True)
label_dir.mkdir(parents=True, exist_ok=True)

img_path = '/content/drive/MyDrive/Final_project/image_detect/Front_data/train/images/0002_03_F.jpg'

img = cv2.imread(img_path)
# BGR 이미지를 RGB로 변환
image_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


# 이미지 탐지
results = model(image_rgb)

# 결과 시각화
# results.print()  # 탐지 결과 출력
# results.show()   # 탐지된 이미지 시각화

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-15 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20885262 parameters, 0 gradients, 48.0 GFLOPs
Adding AutoShape... 


데이터셋 만들기 (부위별 이미지 + 수치값)

부위 이미지 자르기

- 0: 0,  # 얼굴 전체
- 1: 1,  # 이마
- 2: 2,  # 미간
- 3: 3,  # 왼쪽 눈가
- 4: 4,  # 오른쪽 눈가
- 5: 5,  # 왼쪽 볼
- 6: 6,  # 오른쪽 볼
- 7: 7,  # 입술
- 8: 8,  # 턱

- 클래스 id만 바꿔서 사용하세요!

In [10]:
import os
import cv2
import pandas as pd
from pathlib import Path

img_dir = '/content/drive/MyDrive/Final_project/image_detect/Front_data/val/images'
txt_dir = '/content/drive/MyDrive/Final_project/image_detect/Front_data/val/labels'
crop_dir = '/content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/r_peroucular'

#디렉토리 생성
Path(crop_dir).mkdir(parents=True, exist_ok=True)

#txt파일에 있는 바운딩박스 수치값과 클래스 id를 이용해서 뽑기
for txt_file in os.listdir(txt_dir):
    if txt_file.endswith('.txt'):
        img_file = txt_file.replace('.txt', '.jpg')
        img_path = os.path.join(img_dir, img_file)

        #이미지 로드
        img = cv2.imread(img_path)
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue

        img_height, img_width = img.shape[:2]

        #txt파일에서 클래스 1 (이마) 추출
        with open(os.path.join(txt_dir, txt_file), 'r') as f:
            lines = f.readlines()
            for line in lines:
                class_id, x_center, y_center, width, height = map(float, line.split())
                if int(class_id) == 4:  # 클래스 ID가 1인 경우 (이마) --> 이부분 바꾸면 되요
                    # 바운딩 박스 좌표 변환 (YOLO 형식에서 원래 이미지 형식으로 변환)
                    xmin = int((x_center - width / 2) * img_width)
                    ymin = int((y_center - height / 2) * img_height)
                    xmax = int((x_center + width / 2) * img_width)
                    ymax = int((y_center + height / 2) * img_height)

                #자르기 (부위에 맞는 이름으로 변경하세요)
                    r_peroucular_img = img[ymin:ymax, xmin:xmax]
                    cropped_img_path = os.path.join(crop_dir, f"{img_file}_r_peroucular.jpg")
                    cv2.imwrite(cropped_img_path, r_peroucular_img)

                    print(f'Saved cropped image: {cropped_img_path}')

print("All _r_peroucular images have been processed and saved.")


Saved cropped image: /content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/r_peroucular/0013_03_F.jpg_r_peroucular.jpg
Saved cropped image: /content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/r_peroucular/0004_03_F.jpg_r_peroucular.jpg
Saved cropped image: /content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/r_peroucular/0061_03_F.jpg_r_peroucular.jpg
Saved cropped image: /content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/r_peroucular/0001_03_F.jpg_r_peroucular.jpg
Saved cropped image: /content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/r_peroucular/0050_03_F.jpg_r_peroucular.jpg
Saved cropped image: /content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/r_peroucular/0021_03_F.jpg_r_peroucular.jpg
Saved cropped image: /content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/r_peroucular/0781_03_F.jpg_r_

수치데이터 분리 (train/val)

In [15]:
path  = '/content/drive/MyDrive/Final_project/Resnet/train_val_dataset.csv'

df = pd.read_csv(path, index_col=0)

In [16]:
df.head(4)

,ID,elasticity_l,elasticity_r,elasticity_f,elasticity_chin,...,moisture_l,moisture_chin,spot_front,pore_r,pore_l
0,1,0.631136,0.589192,0.696966,0.575362,...,0.646319,0.784773,0.403077,0.244940,0.254942
1,2,0.621871,0.574661,0.573213,0.639686,...,0.532748,0.614340,0.436923,0.279223,0.255766
2,3,0.796368,0.717762,0.523307,0.562542,...,0.467252,0.502287,0.049231,0.166047,0.124382
3,4,0.448989,0.374125,0.605622,0.628399,...,0.410532,0.578423,0.846154,0.375465,0.365321


In [17]:
#df의 ID번호가 train에 있는 image 파일 이름에 포함된 id뽑기
import pandas as pd

df['ID_str']  = df['ID'].apply(lambda x : f"{int(x):04d}")




,ID,elasticity_l,elasticity_r,elasticity_f,elasticity_chin,...,moisture_chin,spot_front,pore_r,pore_l,ID_str
0,1,0.631136,0.589192,0.696966,0.575362,...,0.784773,0.403077,0.244940,0.254942,0001
1,2,0.621871,0.574661,0.573213,0.639686,...,0.614340,0.436923,0.279223,0.255766,0002
2,3,0.796368,0.717762,0.523307,0.562542,...,0.502287,0.049231,0.166047,0.124382,0003
3,4,0.448989,0.374125,0.605622,0.628399,...,0.578423,0.846154,0.375465,0.365321,0004
4,6,0.570992,0.612532,0.546132,0.500734,...,0.672585,0.227692,0.118133,0.153213,0006


In [18]:
df.tail()

,ID,elasticity_l,elasticity_r,elasticity_f,elasticity_chin,...,moisture_chin,spot_front,pore_r,pore_l,ID_str
1067,1096,0.625066,0.550603,0.635144,0.654714,...,0.847458,0.206154,0.171004,0.132619,1096
1068,1097,0.305446,0.510375,0.670983,0.704331,...,0.654695,0.095385,0.084676,0.072900,1097
1069,1098,0.506551,0.552128,0.640444,0.592678,...,0.852031,0.153846,0.273854,0.249588,1098
1070,1099,0.566444,0.563007,0.650883,0.705329,...,0.789212,0.086154,0.144568,0.107496,1099
1071,1100,0.495638,0.507562,0.518440,0.795545,...,0.551520,0.375385,0.530359,0.381796,1100


In [25]:
df.columns

Index(['ID', 'elasticity_l', 'elasticity_r', 'elasticity_f', 'elasticity_chin',
       'wrinkle_r', 'wrinkle_l', 'moisture_f', 'moisture_r', 'moisture_l',
       'moisture_chin', 'spot_front', 'pore_r', 'pore_l', 'ID_str'],
      dtype='object')

train/val 에 하당하는 id 데이터 나누기

In [33]:
import os
import pandas as pd

# 이미지 파일이 담긴 폴더 경로
image_dir = '/content/drive/MyDrive/Final_project/image_detect/Front_data/val/cropped_images/forehead'

# 이미지 파일 리스트 가져오기
image_files = os.listdir(image_dir)

# 이미지 파일명에서 숫자 추출
image_ids = [int(f.split('_')[0]) for f in image_files]


# 필요한 행만 필터링 (id 컬럼이 image_ids에 포함된 경우)
filtered_df = df[df['ID'].isin(image_ids)]

# 결과 출력
print(filtered_df)

# 필요한 경우, 필터링된 데이터를 별도의 CSV 파일로 저장할 수 있습니다.
filtered_df.to_csv('/content/drive/MyDrive/Final_project/image_detect/Front_data/val/val_filtered_data.csv', index=False)


        ID  elasticity_l  elasticity_r  elasticity_f  elasticity_chin  ...  \
0        1      0.631136      0.589192      0.696966         0.575362  ...   
3        4      0.448989      0.374125      0.605622         0.628399  ...   
11      13      0.532322      0.387143      0.646682         0.732237  ...   
19      21      0.598548      0.636141      0.565445         0.372159  ...   
48      50      0.529154      0.440773      0.510576         0.553984  ...   
...    ...           ...           ...           ...              ...  ...   
1031  1060      0.509440      0.477356      0.451131         0.676978  ...   
1036  1065      0.617774      0.634626      0.673799         0.940004  ...   
1047  1076      0.581379      0.604109      0.691104         0.617322  ...   
1052  1081      0.644977      0.645098      0.625144         0.727485  ...   
1062  1091      0.409204      0.439000      0.568708         0.571496  ...   

      moisture_chin  spot_front    pore_r    pore_l  ID_str  
0